In [ ]:
# Install requirements
!pip install torchinfo>=1.8.0 numpy>=1.24.0 pandas>=2.0.0 scikit-learn>=1.4.0,<1.5.0 imbalanced-learn>=0.12.0 torch torchvision matplotlib>=3.7.0 seaborn>=0.12.0 tqdm>=4.65.0 psutil>=5.9.0

In [ ]:
# ==========================================
# CONTENT FROM: config.py
# ==========================================
import os
import torch

# =============================================================================
# SET WORKING DIRECTORY
# =============================================================================
WORKING_DIR = '.' 
os.makedirs(WORKING_DIR, exist_ok=True)

# =============================================================================
# SYSTEM RESOURCES & DEVICE
# =============================================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =============================================================================
# CONFIGURATION
# =============================================================================
BATCH_SIZE = 256
N_WORKERS = 8 # DataLoader Workers

# Check RAM (optional logic can be here or main)
import psutil
ram_gb = psutil.virtual_memory().total / 1e9

# Training config
N_EPOCHS = 6
LEARNING_RATE = 0.0001
IMAGE_SIZE = 32

# =============================================================================
# DEFINING SCENARIOS
# =============================================================================
# Train scenarios: 
TRAIN_SCENARIOS = ['11']

# Test scenario: Rbot
TEST_SCENARIOS = ['10']

# =============================================================================
# LABEL MAPPING
# =============================================================================
CLASS_TO_IDX = {
    'Botnet': 0,
    'C&C': 1,
    'Normal': 2
}

# Inverse mapping
IDX_TO_CLASS = {v: k for k, v in CLASS_TO_IDX.items()}


In [ ]:
# ==========================================
# CONTENT FROM: utils.py
# ==========================================
import os
import ssl
import urllib.request
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

# Bypass SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

def create_directory(path):
    os.makedirs(path, exist_ok=True)
    print(f"  Created/Checked: {path}")

def download_file(url, destination):
    try:
        if os.path.exists(destination):
            print(f"  [SKIP] File exists: {os.path.basename(destination)}")
            return True

        print(f"  Downloading: {os.path.basename(destination)}")
        with tqdm(unit='B', unit_scale=True, unit_divisor=1024, miniters=1, desc="  Progress") as t:
            def reporthook(blocknum, blocksize, totalsize):
                t.total = totalsize
                t.update(blocknum * blocksize - t.n)
            urllib.request.urlretrieve(url, destination, reporthook=reporthook)
        return True
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")
        return False

def check_csv_in_folder(folder_path):
    if not os.path.exists(folder_path): return False
    for file in os.listdir(folder_path):
        if file.endswith('.csv'): return True
    return False

def rename(path_file, new_name):
    dir_path = os.path.dirname(path_file)
    path_new_name = os.path.join(dir_path, new_name)
    os.rename(path_file, path_new_name)

def get_csv_paths(main_dir, scenario_ids):
    csv_paths = []
    for sid in scenario_ids:
        path = os.path.join(main_dir, sid)
        # Find csv file in folder
        if os.path.exists(path):
            for file in os.listdir(path):
                if file.endswith('.csv'):
                    csv_paths.append(os.path.join(path, file))
                    break
    return csv_paths

def plot_and_save_loss(train_losses, valid_losses, save_path):
    """
    Plots Loss chart and saves to file.
    """
    plt.figure(figsize=(10, 6))

    # Plot Train Loss
    plt.plot(train_losses, label='Train Loss', color='blue', marker='o', markersize=4)

    # Plot Valid Loss
    plt.plot(valid_losses, label='Valid Loss', color='orange', marker='o', markersize=4)

    # Decorate
    plt.title('Training vs Validation Loss', fontsize=16)
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)

    # Save image
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"\n[INFO] Saved Loss plot at: {save_path}")

    # Show (optional in script mode, but harmless)
    # plt.show() 
    plt.close()


In [ ]:
# ==========================================
# CONTENT FROM: preprocessing_utils.py
# ==========================================
import pandas as pd
import numpy as np
import pickle
import os
from tqdm.auto import tqdm

def quick_classify(label):
    """
    Classifies a label string into 'Botnet', 'C&C', or 'Normal'.
    """
    if not isinstance(label, str):
        return 'Normal'
    label_lower = label.lower()
    if 'c&c' in label_lower or 'cc' in label_lower:
        return 'C&C'
    elif 'botnet' in label_lower:
        return 'Botnet'
    else:
        return 'Normal'

def calculate_global_frequencies(csv_paths):
    """
    Calculates global frequencies for IPs and Ports across all CSV files.
    Returns a dictionary of dictionaries.
    """
    freqs = {
        'SrcAddr': {},
        'DstAddr': {},
        'Sport': {},
        'Dport': {}
    }
    
    print("Scanning CSVs for frequencies...")
    for path in tqdm(csv_paths, desc="Global Freqs"):
        try:
            # Read specific columns to save memory
            # Note: low_memory=False helps with mixed types
            df = pd.read_csv(path, usecols=['SrcAddr', 'DstAddr', 'Sport', 'Dport'], low_memory=False)
            
            for col in freqs:
                if col in df.columns:
                    # Value counts
                    vc = df[col].value_counts().to_dict()
                    for k, v in vc.items():
                        # We use simple types for keys to ensure matches later
                        # Convert key to string if needed?
                        # Ports might be int or str. IPs are str.
                        # Let's keep original types but be careful in mapping.
                        # Actually, better to normalize to string for consistency?
                        # But process_batch converts to numeric for ports.
                        # Let's stick to raw values here, assume map handles it or we align later.
                        # But for safety, maybe convert to string?
                        # The notebook output says "Unique Source Ports: 61941".
                        
                        freqs[col][k] = freqs[col].get(k, 0) + v
        except Exception as e:
            print(f"Warning: Could not process {path} for frequencies: {e}")
            
    return freqs

def process_batch_fast_v2(chunk, freq_dicts, expected_columns=None):
    """
    Processes a batch (DataFrame chunk).
    1. Cleans data.
    2. Encodes features (Frequencies, One-Hot).
    3. Aligns columns.
    Returns (X_values, y_values, columns_list).
    """
    df = chunk.copy()
    
    # 1. Label Processing
    if 'Label' in df.columns:
        y = df['Label'].apply(quick_classify)
        df = df.drop(columns=['Label'])
    else:
        y = None 

    # 2. Basic Cleaning
    if 'StartTime' in df.columns:
        df = df.drop(columns=['StartTime'])
        
    # 3. Numeric Conversions
    # Columns that should be numeric
    numeric_cols = ['Dur', 'TotPkts', 'TotBytes', 'SrcBytes']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
            
    # 4. Feature Engineering: Frequencies
    for col in ['SrcAddr', 'DstAddr', 'Sport', 'Dport']:
        freq_col_name = f"{col}_freq"
        if col in df.columns and col in freq_dicts:
            # Map values. Using map with a dict. 
            # Note: If types don't match (e.g. int vs str), map produces NaN.
            # We assume consistent types from read_csv.
            df[freq_col_name] = df[col].map(freq_dicts.get(col, {})).fillna(0)
        else:
            df[freq_col_name] = 0

    # 5. Split IP thành 4 octet (4 feature riêng) thay vì một integer lớn
    for ip_col in ['SrcAddr', 'DstAddr']:
        if ip_col in df.columns:
            # Chuyển sang string để split theo '.'
            ip_str = df[ip_col].astype(str)
            parts = ip_str.str.split('.', expand=True)

            # Đảm bảo có đủ 4 cột (nếu IP không chuẩn, phần thiếu sẽ là NaN)
            for i in range(4):
                if i < parts.shape[1]:
                    df[f'{ip_col}_octet_{i+1}'] = pd.to_numeric(parts[i], errors='coerce').fillna(0)
                else:
                    # Nếu thiếu cột, tạo cột toàn 0
                    df[f'{ip_col}_octet_{i+1}'] = 0

    # 6. Handle Port Columns (Keep as numeric features)
    for col in ['Sport', 'Dport']:
        if col in df.columns:
            # Force numeric (handle hex or strings)
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    # 7. Drop raw IP addresses (đã có tần suất + 4 octet)
    df = df.drop(columns=['SrcAddr', 'DstAddr'], errors='ignore')
    
    # 8. One-Hot Encoding for 'Proto'
    if 'Proto' in df.columns:
        # Common protos: TCP, UDP, ICMP
        # Limit cardinality if needed? Usually low.
        dummies = pd.get_dummies(df['Proto'], prefix='Proto')
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(columns=['Proto'])
        
    # 8. One-Hot Encoding for 'State' (không giới hạn top state)
    if 'State' in df.columns:
        dummies = pd.get_dummies(df['State'], prefix='State')
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(columns=['State'])
        
    # 9. Handle 'Dir'
    if 'Dir' in df.columns:
         dummies = pd.get_dummies(df['Dir'], prefix='Dir')
         df = pd.concat([df, dummies], axis=1)
         df = df.drop(columns=['Dir'])
         
    # 10. Align columns with expected_columns
    if expected_columns is not None:
        # Add missing columns with 0
        # Use reindex for efficiency and handling both missing and extra
        df = df.reindex(columns=expected_columns, fill_value=0)
    else:
        # First time detection: Fill NaNs just in case
        df = df.fillna(0)
        
    # Return values
    X_vals = df.values.astype(np.float32) # Ensure float32 for PyTorch
    y_vals = y.values if y is not None else None
    
    return X_vals, y_vals, df.columns.tolist()

def save_global_stats(global_stats, filepath='global_stats.pkl'):
    """Saves global statistics to a pickle file."""
    with open(filepath, 'wb') as f:
        pickle.dump(global_stats, f)
    print(f"Global stats saved to {filepath}")

def load_global_stats(filepath='global_stats.pkl'):
    """Loads global statistics from a pickle file."""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"{filepath} not found.")
    with open(filepath, 'rb') as f:
        return pickle.load(f)

def save_scaler(scaler, filepath='scaler.pkl'):
    """Saves the scaler to a pickle file."""
    with open(filepath, 'wb') as f:
        pickle.dump(scaler, f)
    print(f"Scaler saved to {filepath}")

def load_scaler(filepath='scaler.pkl'):
    """Loads the scaler from a pickle file."""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"{filepath} not found.")
    with open(filepath, 'rb') as f:
        return pickle.load(f)


In [ ]:
# ==========================================
# CONTENT FROM: loss.py
# ==========================================
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=2., reduction='mean'):
        """
        Args:
            weight (Tensor, optional): A manual rescaling weight given to each class.
                                       If given, has to be a Tensor of size C.
            gamma (float): Focusing parameter.
            reduction (string, optional): Specifies the reduction to apply to the output:
                                          'none' | 'mean' | 'sum'. 'mean': the sum of the output will be divided by the number of elements in the output,
                                          'sum': the output will be summed. Default: 'mean'
        """
        super(FocalLoss, self).__init__()
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        Args:
            inputs: (N, C) where C = number of classes.
            targets: (N) where each value is 0 <= targets[i] <= C-1.
        """
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


In [ ]:
# ==========================================
# CONTENT FROM: model.py
# ==========================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models


class BotnetImageCNN(nn.Module):
    """
    CNN 2D dùng pretrained MobileNetV3-Small.
    - Input: ảnh 1x32x32
    - Nội bộ resize lên 224x224 cho phù hợp với pretrained weights.
    """

    def __init__(self, n_classes: int = 3):
        super().__init__()

        # Pretrained MobileNetV3 Small (nhẹ)
        weights = models.MobileNet_V3_Small_Weights.DEFAULT
        base = models.mobilenet_v3_small(weights=weights)

        # Chỉnh conv đầu tiên nhận 1 kênh thay vì 3 kênh
        first_conv = base.features[0][0]
        new_conv = nn.Conv2d(
            in_channels=1,
            out_channels=first_conv.out_channels,
            kernel_size=first_conv.kernel_size,
            stride=first_conv.stride,
            padding=first_conv.padding,
            bias=False,
        )

        # Khởi tạo trọng số từ pretrained: trung bình theo channel
        with torch.no_grad():
            new_conv.weight[:] = first_conv.weight.mean(dim=1, keepdim=True)

        base.features[0][0] = new_conv

        # Thay classifier cuối cho đúng số lớp
        in_features = base.classifier[-1].in_features
        base.classifier[-1] = nn.Linear(in_features, n_classes)

        self.base = base

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: [B, 1, 32, 32] -> resize 224x224 cho MobileNet
        x = F.interpolate(x, size=(224, 224), mode="bilinear", align_corners=False)
        return self.base(x)


class BotnetClassifier(nn.Module):
    """
    Wrapper giữ API cũ:
      - vẫn nhận n_features, image_size nhưng bỏ qua; dữ liệu đã là ảnh 1x32x32.
    """

    def __init__(self, base_model=None, n_features=None, image_size: int = 32, n_classes: int = 3):
        super().__init__()
        self.model = BotnetImageCNN(n_classes=n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)


In [ ]:
# ==========================================
# CONTENT FROM: data_loader.py
# ==========================================
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# from preprocessing_utils import process_batch_fast_v2  # Commented out for notebook compatibility
# from config import CLASS_TO_IDX, IMAGE_SIZE  # Commented out for notebook compatibility


class FastBotnetDataset(Dataset):
    """
    Dataset:
      - Nhận X_data dạng [N, n_features]
      - Mỗi dòng được pad / cắt về 32x32 và reshape thành ảnh 1x32x32
    """

    def __init__(self, X_data, y_data):
        self.X_data = torch.from_numpy(X_data).float()
        self.y_data = torch.from_numpy(y_data).long()

        self.image_size = IMAGE_SIZE
        self.num_pixels = self.image_size * self.image_size

        # Pre-compute Hilbert curve coordinates for mapping 1D features -> 2D ảnh
        self.hilbert_coords = self._hilbert_curve(self.image_size)

    def __len__(self):
        return len(self.X_data)

    def _hilbert_curve(self, n):
        """
        Sinh danh sách tọa độ (row, col) theo đường cong Hilbert cho lưới n x n.
        n phải là lũy thừa của 2 (ở đây n = 32).
        Thuật toán đơn giản, ưu tiên tính dễ đọc hơn là tối ưu tuyệt đối.
        """
        if n & (n - 1) != 0:
            raise ValueError("IMAGE_SIZE must be a power of 2 for Hilbert curve.")

        def hilbert_index_to_xy(d, order):
            """
            Chuyển chỉ số 1D d trên Hilbert order 'order' (2^order x 2^order)
            sang tọa độ (x, y).
            """
            x = y = 0
            t = d
            s = 1
            while s < (1 << order):
                rx = 1 & (t // 2)
                ry = 1 & (t ^ rx)
                # xoay / phản chiếu
                if ry == 0:
                    if rx == 1:
                        x, y = s - 1 - x, s - 1 - y
                    x, y = y, x
                x += s * rx
                y += s * ry
                t //= 4
                s <<= 1
            return x, y

        order = int(np.log2(n))
        coords = []
        for d in range(n * n):
            x, y = hilbert_index_to_xy(d, order)
            coords.append((x, y))
        return coords

    def __getitem__(self, idx):
        # vector đặc trưng 1D: [n_features]
        feature_vector = self.X_data[idx]

        # Khởi tạo ảnh 0 (0..255), sẽ điền theo đường cong Hilbert
        image = torch.zeros((1, self.image_size, self.image_size), dtype=feature_vector.dtype)

        # Số feature thực sự có (không bỏ dòng nào, chỉ giới hạn số pixel)
        length = min(feature_vector.numel(), self.num_pixels)

        # Map từng feature lên pixel theo Hilbert curve để giữ locality
        for i in range(length):
            x, y = self.hilbert_coords[i]
            image[0, x, y] = feature_vector[i]

        # Scale per-sample về khoảng [0, 255]
        vals = image.view(-1)
        vals = torch.clamp(vals, min=0)
        min_val = vals.min()
        max_val = vals.max()
        if max_val > min_val:
            vals = (vals - min_val) / (max_val - min_val)
        else:
            vals = vals * 0.0
        vals = vals * 255.0
        image = vals.view(1, self.image_size, self.image_size)

        return image, self.y_data[idx]

def load_data_from_csvs(csv_list, global_stats, desc="Loading", is_train=True, scaler=None):
    """
    Loads data from a list of CSVs, processes it, and returns X and y.
    """
    X_list = []
    y_list = []

    # Get stats
    freq_dicts = global_stats['freq_dicts']
    expected_cols = global_stats['expected_columns']

    for csv_file in tqdm(csv_list, desc=desc):
        try:
            for chunk in pd.read_csv(csv_file, chunksize=100000, low_memory=False):
                X_batch, y_batch, _ = process_batch_fast_v2(
                    chunk, freq_dicts, expected_cols
                )
                if len(X_batch) > 0:
                    X_list.append(X_batch)
                    # Convert labels to indices
                    # y_batch is a Series or numpy array of strings
                    if y_batch is not None:
                         # Ensure y_batch contains valid keys from CLASS_TO_IDX
                        y_indices = np.array([CLASS_TO_IDX.get(label, 2) for label in y_batch]) # Default to Normal (2) if not found
                        y_list.append(y_indices)
        except Exception as e:
            print(f"  Error loading {csv_file}: {e}")

    if not X_list:
        return np.array([]), np.array([])

    X_data = np.vstack(X_list).astype(np.float32)
    y_data = np.concatenate(y_list).astype(np.int64)
    # 1. Sanitize input: Replace existing NaNs/Infs with 0
    X_data = np.nan_to_num(X_data, nan=0.0, posinf=0.0, neginf=0.0)

    # 2. Clip negative values to 0. 
    # This prevents np.log1p(-1) from becoming -inf.
    X_data = np.maximum(X_data, 0)

    # Normalize (Log1p + Scaler)
    X_data = np.log1p(X_data)

    if scaler:
        if is_train:
            X_data = scaler.fit_transform(X_data)
        else:
            X_data = scaler.transform(X_data)

    return X_data, y_data


In [ ]:
# ==========================================
# CONTENT FROM: analyze_csv.py
# ==========================================
import os
import argparse
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from preprocessing_utils import quick_classify  # Commented out for notebook compatibility

# Set style for plots
sns.set_theme(style="whitegrid")
# Ensure matplotlib can display Vietnamese characters if supported font is found, 
# otherwise fallback to default. 
# In a standard env, this might be tricky, so we stick to standard fonts but write strings in Vietnamese.

def load_data(filepath):
    """Loads CSV data."""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Không tìm thấy file: {filepath}")
    
    # Check if empty
    if os.path.getsize(filepath) == 0:
         raise ValueError(f"File rỗng: {filepath}")

    print(f"Đang tải dữ liệu từ: {filepath} ...")
    # Low memory false to handle mixed types if any, similar to training script
    df = pd.read_csv(filepath, low_memory=False)
    return df

def analyze_data(df):
    """Calculates statistics and metrics."""
    stats = {}
    
    # 1. Label Mapping
    if 'Label' in df.columns:
        df['Mapped_Label'] = df['Label'].apply(quick_classify)
        
        # Raw Label Counts
        stats['raw_label_counts'] = df['Label'].value_counts().head(20).to_dict()
        
        # Mapped Label Counts
        stats['mapped_label_counts'] = df['Mapped_Label'].value_counts().to_dict()
        stats['mapped_label_pct'] = df['Mapped_Label'].value_counts(normalize=True).to_dict()
    else:
        stats['raw_label_counts'] = {}
        stats['mapped_label_counts'] = {}
        print("CẢNH BÁO: Không tìm thấy cột 'Label'.")

    # 2. Numerical Features Stats
    num_cols = ['Dur', 'TotPkts', 'TotBytes', 'SrcBytes']
    stats['numerical'] = {}
    
    for col in num_cols:
        if col in df.columns:
            # Force numeric
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
            desc = df[col].describe()
            stats['numerical'][col] = {
                'mean': desc['mean'],
                'std': desc['std'],
                'min': desc['min'],
                'max': desc['max'],
                'median': desc['50%']
            }
        else:
            stats['numerical'][col] = None

    # 3. Categorical Counts
    cat_cols = ['Proto', 'State', 'Dir']
    stats['categorical'] = {}
    for col in cat_cols:
        if col in df.columns:
            stats['categorical'][col] = df[col].value_counts().head(10).to_dict()
        else:
            stats['categorical'][col] = {}
            
    # 4. Missing Values
    stats['missing_values'] = df.isnull().sum().to_dict()
    
    return stats, df

def generate_report(stats, df, output_dir, filename_base):
    """Generates text report and plots."""
    
    # --- 1. Text Report ---
    report_path = os.path.join(output_dir, f"{filename_base}_report.txt")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(f"BÁO CÁO PHÂN TÍCH DỮ LIỆU: {filename_base}\n")
        f.write("="*60 + "\n\n")
        
        f.write("1. TỔNG QUAN\n")
        f.write(f"   - Tổng số dòng: {len(df)}\n")
        f.write(f"   - Số cột: {len(df.columns)}\n")
        f.write("\n")
        
        f.write("2. PHÂN BỐ NHÃN (Lớp Mapped)\n")
        if stats['mapped_label_counts']:
            for label, count in stats['mapped_label_counts'].items():
                pct = stats['mapped_label_pct'].get(label, 0) * 100
                f.write(f"   - {label}: {count} ({pct:.2f}%)\n")
        else:
            f.write("   (Không có dữ liệu nhãn)\n")
        f.write("\n")
        
        f.write("3. TOP 20 NHÃN GỐC (Raw Labels)\n")
        if stats['raw_label_counts']:
            for label, count in stats['raw_label_counts'].items():
                f.write(f"   - {label}: {count}\n")
        f.write("\n")
        
        f.write("4. THỐNG KÊ SỐ HỌC (Numerical Stats)\n")
        for col, data in stats['numerical'].items():
            if data:
                f.write(f"   * {col}:\n")
                f.write(f"     - Trung bình (Mean): {data['mean']:.2f}\n")
                f.write(f"     - Trung vị (Median): {data['median']:.2f}\n")
                f.write(f"     - Lớn nhất (Max):    {data['max']:.2f}\n")
                f.write(f"     - Độ lệch chuẩn (Std): {data['std']:.2f}\n")
            else:
                f.write(f"   * {col}: Không tìm thấy\n")
        f.write("\n")

        f.write("5. THÔNG TIN PHÂN LOẠI (Categorical Info)\n")
        for col, data in stats['categorical'].items():
            f.write(f"   * Top 10 {col}:\n")
            for k, v in data.items():
                f.write(f"     - {k}: {v}\n")
        f.write("\n")

        f.write("6. GIÁ TRỊ THIẾU (Missing Values)\n")
        for col, count in stats['missing_values'].items():
            if count > 0:
                f.write(f"   - {col}: {count}\n")
    
    print(f"Đã lưu báo cáo văn bản tại: {report_path}")

    # --- 2. Visualization ---
    image_path = os.path.join(output_dir, f"{filename_base}_visuals.png")
    
    # Setup Figure: 2 Rows, 3 Columns
    fig = plt.figure(figsize=(20, 12), constrained_layout=True)
    gs = fig.add_gridspec(2, 3)

    # A. Class Distribution (Mapped)
    ax1 = fig.add_subplot(gs[0, 0])
    if 'Mapped_Label' in df.columns:
        sns.countplot(x='Mapped_Label', data=df, ax=ax1, palette='viridis', hue='Mapped_Label', legend=False)
        ax1.set_title("Phân bố Lớp (Mapped Classes)", fontsize=14)
        ax1.set_ylabel("Số lượng")
        ax1.set_xlabel("Lớp")
    else:
        ax1.text(0.5, 0.5, "Không có dữ liệu nhãn", ha='center')

    # B. Numerical Distributions (Boxplots Log Scale)
    ax2 = fig.add_subplot(gs[0, 1])
    # Melting for easier plotting of multiple features
    num_cols = ['Dur', 'TotPkts', 'TotBytes', 'SrcBytes']
    # Filter valid columns
    valid_num = [c for c in num_cols if c in df.columns]
    
    if valid_num:
        # We take a sample if data is too huge to plot quickly? No, boxplot is fast enough usually.
        # But we need log scale because Bytes can be huge.
        df_melt = df[valid_num].melt(var_name='Feature', value_name='Value')
        # Log transformation for display (handling 0)
        df_melt['LogValue'] = np.log1p(df_melt['Value'])
        
        sns.boxplot(x='Feature', y='LogValue', data=df_melt, ax=ax2, hue='Feature', palette="Set2")
        ax2.set_title("Phân bố Đặc trưng Số (Log Scale)", fontsize=14)
        ax2.set_ylabel("Log(Giá trị + 1)")
    else:
        ax2.text(0.5, 0.5, "Không có dữ liệu số", ha='center')

    # C. Top Raw Labels (Horizontal Bar)
    ax3 = fig.add_subplot(gs[0, 2])
    if 'Label' in df.columns:
        top_labels = df['Label'].value_counts().head(10)
        sns.barplot(y=top_labels.index, x=top_labels.values, ax=ax3, palette="magma", hue=top_labels.index, legend=False)
        ax3.set_title("Top 10 Nhãn Gốc (Raw Labels)", fontsize=14)
        ax3.set_xlabel("Số lượng")
    else:
        ax3.text(0.5, 0.5, "Không có dữ liệu nhãn", ha='center')

    # D. Protocol Distribution
    ax4 = fig.add_subplot(gs[1, 0])
    if 'Proto' in df.columns:
        top_proto = df['Proto'].value_counts().head(10)
        sns.barplot(x=top_proto.index, y=top_proto.values, ax=ax4, palette="Blues_d", hue=top_proto.index, legend=False)
        ax4.set_title("Top Giao thức (Protocol)", fontsize=14)
        ax4.set_ylabel("Số lượng")
    else:
        ax4.text(0.5, 0.5, "Không có Proto", ha='center')

    # E. State Distribution
    ax5 = fig.add_subplot(gs[1, 1])
    if 'State' in df.columns:
        top_state = df['State'].value_counts().head(10)
        sns.barplot(x=top_state.index, y=top_state.values, ax=ax5, palette="Reds_d", hue=top_state.index, legend=False)
        ax5.set_title("Top Trạng thái (State)", fontsize=14)
        ax5.set_ylabel("Số lượng")
        ax5.tick_params(axis='x', rotation=45)
    else:
        ax5.text(0.5, 0.5, "Không có State", ha='center')

    # F. Feature by Class (e.g., TotBytes per Class)
    ax6 = fig.add_subplot(gs[1, 2])
    if 'Mapped_Label' in df.columns and 'TotBytes' in df.columns:
         # Use LogBytes
         df['LogTotBytes'] = np.log1p(df['TotBytes'])
         sns.boxplot(x='Mapped_Label', y='LogTotBytes', data=df, ax=ax6, palette="coolwarm", hue='Mapped_Label', legend=False)
         ax6.set_title("TotBytes theo Lớp (Log Scale)", fontsize=14)
         ax6.set_ylabel("Log(TotBytes)")
    else:
        ax6.text(0.5, 0.5, "Thiếu dữ liệu để vẽ biểu đồ", ha='center')

    # Save
    plt.suptitle(f"Dashboard Phân Tích: {filename_base}", fontsize=20)
    plt.savefig(image_path)
    plt.close()
    
    print(f"Đã lưu biểu đồ tại: {image_path}")


def analyze_csv_main():
    parser = argparse.ArgumentParser(description="Script phân tích file CSV CTU-13")
    parser.add_argument("--file", type=str, required=True, help="Đường dẫn đến file .csv")
    parser.add_argument("--outdir", type=str, default="analysis_reports", help="Thư mục lưu kết quả")
    
    args = parser.parse_args()
    
    # Create output dir
    os.makedirs(args.outdir, exist_ok=True)
    
    # Filename base
    base_name = os.path.splitext(os.path.basename(args.file))[0]
    # To avoid overwriting if same filename exists in different folders, maybe prepend parent folder?
    # e.g. 10_10.csv
    parent_dir = os.path.basename(os.path.dirname(args.file))
    if parent_dir.isdigit():
        base_name = f"{parent_dir}_{base_name}"
    
    try:
        df = load_data(args.file)
        stats, df_processed = analyze_data(df)
        generate_report(stats, df_processed, args.outdir, base_name)
        print("Hoàn tất!")
    except Exception as e:
        print(f"LỖI: {e}")

# if __name__ == "__main__": # Block disabled for notebook import
#     analyze_csv_main()


In [ ]:
# ==========================================
# CONTENT FROM: train.py
# ==========================================
import os
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.utils import class_weight
from collections import Counter
from imblearn.over_sampling import SMOTE
from tqdm.auto import tqdm
# from loss import FocalLoss  # Commented out for notebook compatibility
try:
    from torchinfo import summary
except ImportError:
    summary = None

# from config import (  # Commented out for notebook compatibility
    WORKING_DIR, BATCH_SIZE, N_WORKERS, N_EPOCHS, LEARNING_RATE,
    IMAGE_SIZE, TRAIN_SCENARIOS, TEST_SCENARIOS,
    CLASS_TO_IDX, device
)
# from utils import (  # Commented out for notebook compatibility
    create_directory, download_file, check_csv_in_folder, 
    rename, get_csv_paths, plot_and_save_loss
)
# from preprocessing_utils import (  # Commented out for notebook compatibility
    quick_classify, calculate_global_frequencies, process_batch_fast_v2,
    save_global_stats, save_scaler
)
# from model import BotnetClassifier  # Commented out for notebook compatibility
# from data_loader import FastBotnetDataset, load_data_from_csvs  # Commented out for notebook compatibility

def train_main(ispart=True):
    print(f"Working directory: {os.getcwd()}")
    print("="*60)
    print("SYSTEM RESOURCES")
    print("="*60)
    
    if device.type == 'cuda':
        print(f"  GPU: {torch.cuda.get_device_name(0)}")
        
    print(f"\n" + "="*60)
    print("CONFIGURATION")
    print("="*60)
    print(f"  Batch Size: {BATCH_SIZE}")
    print(f"  Epochs: {N_EPOCHS}")
    print(f"  Learning Rate: {LEARNING_RATE}")
    print(f"  DataLoader Workers: {N_WORKERS}")
    print("="*60)

    # =============================================================================
    # 1. DOWNLOAD DATASET
    # =============================================================================
    main_dir = './CTU-13-Dataset/'
    create_directory(main_dir)

    for i in range(1, 14):
        create_directory(os.path.join(main_dir, str(i)))

    datasets = [
        (1, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-42/detailed-bidirectional-flow-labels/capture20110810.binetflow'),
        (2, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-43/detailed-bidirectional-flow-labels/capture20110811.binetflow'),
        (3, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-44/detailed-bidirectional-flow-labels/capture20110812.binetflow'),
        (4, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-45/detailed-bidirectional-flow-labels/capture20110815.binetflow'),
        (5, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-46/detailed-bidirectional-flow-labels/capture20110815-2.binetflow'),
        (6, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-47/detailed-bidirectional-flow-labels/capture20110816.binetflow'),
        (7, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-48/detailed-bidirectional-flow-labels/capture20110816-2.binetflow'),
        (8, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-49/detailed-bidirectional-flow-labels/capture20110816-3.binetflow'),
        (9, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-50/detailed-bidirectional-flow-labels/capture20110817.binetflow'),
        (10, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-51/detailed-bidirectional-flow-labels/capture20110818.binetflow'),
        (11, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-52/detailed-bidirectional-flow-labels/capture20110818-2.binetflow'),
        (12, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-53/detailed-bidirectional-flow-labels/capture20110819.binetflow'),
        (13, 'https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-54/detailed-bidirectional-flow-labels/capture20110815-3.binetflow')
    ]

    print(f"\nStarting download for {len(datasets)} datasets...")
    for idx, url in datasets:
        filename = url.split('/')[-1]
        destination = os.path.join(main_dir, str(idx), filename)
        folder_path = os.path.join(main_dir, str(idx))

        print(f"\n[{idx}/13] Dataset {idx}:")
        if check_csv_in_folder(folder_path):
            print(f"  [SKIP] CSV already exists.")
            continue
        download_file(url, destination)
    print("\nDownload complete!")

    # =============================================================================
    # 2. CONVERT BINETFLOW TO CSV
    # =============================================================================
    listDir = os.listdir(main_dir)
    listCSV = []
    print("Checking/Converting files...")

    for subDir in sorted(listDir, key=lambda x: int(x) if x.isdigit() else 0):
        path_subDir = os.path.join(main_dir, subDir)
        if not os.path.isdir(path_subDir): continue

        # Check if CSV exists
        csv_files = [f for f in os.listdir(path_subDir) if f.endswith('.csv')]
        if csv_files:
            listCSV.append(os.path.join(path_subDir, csv_files[0]))
            continue

        # If no CSV, look for binetflow to rename
        binetflow_files = [f for f in os.listdir(path_subDir) if 'binetflow' in f]
        if binetflow_files:
            binetflow_file = os.path.join(path_subDir, binetflow_files[0])
            new_name = subDir + '.csv'
            rename(binetflow_file, new_name)
            listCSV.append(os.path.join(path_subDir, new_name))
            print(f"  Converted {binetflow_files[0]} -> {new_name}")

    print(f"\nFound {len(listCSV)} CSV files:")
    for csv in listCSV:
        print(f"  {csv}")

    # =============================================================================
    # 3. IDENTIFY SCENARIOS
    # =============================================================================
    train_csvs = get_csv_paths(main_dir, TRAIN_SCENARIOS)
    test_csvs = get_csv_paths(main_dir, TEST_SCENARIOS)
    
    print(f"Found {len(train_csvs)} training files.")
    print(f"Found {len(test_csvs)} testing files.")

    if not train_csvs:
        print("No training files found! Exiting.")
        return

    # =============================================================================
    # 4. PRE-COMPUTE GLOBAL STATISTICS
    # =============================================================================
    print("="*70)
    print("PRE-COMPUTING GLOBAL STATISTICS")
    print("="*70)
    
    target_csvs = train_csvs 

    # --- Step 1: Calculate Global Frequencies ---
    print("\n[1/4] Calculating global IP/Port frequencies...")
    freq_dicts = calculate_global_frequencies(target_csvs)

    # --- Step 2: Detect Column Schema ---
    print("\n[2/3] Detecting column schema...")
    expected_columns = None
    cols_samples = []

    for csv_file in target_csvs[:5]:
        try:
            chunk = pd.read_csv(csv_file, nrows=5000, low_memory=False)
            X_s, y_s, cols_s = process_batch_fast_v2(chunk, freq_dicts, expected_columns=None)
            if cols_s:
                cols_samples.extend(cols_s)
        except Exception:
            continue

    if cols_samples:
        expected_columns = list(dict.fromkeys(cols_samples))  # Preserve order, remove duplicates
        print(f"  Detected {len(expected_columns)} feature columns")
    else:
        print("  WARNING: Could not detect column schema!")
        return

    # --- Step 3: Chuẩn bị global_stats ---
    n_features = len(expected_columns)

    global_stats = {
        'freq_dicts': freq_dicts,
        'expected_columns': expected_columns,
        'n_features': n_features
    }

    # =============================================================================
    # 5. LOAD DATA INTO RAM
    # =============================================================================
    scaler = RobustScaler()
    
    print("\nLoading TRAINING Data...")
    X_train, y_train = load_data_from_csvs(train_csvs, global_stats, desc="Train Data", is_train=True, scaler=scaler)

    # Optionally use only a part of the training set while keeping class ratio
    if ispart:
        subset_fraction = 0.1  # dùng 10% dữ liệu train, vẫn giữ đúng tỉ lệ class
        print(f"\n[ISPART] Using only {subset_fraction*100:.1f}% of training data with stratified sampling...")
        X_train, _, y_train, _ = train_test_split(
            X_train,
            y_train,
            train_size=subset_fraction,
            stratify=y_train,
            random_state=42
        )
        print(f"New Train shape after ISPART: {X_train.shape}")
    
    # Save statistics and scaler after processing training data
    print("\nSaving Global Statistics and Scaler...")
    save_global_stats(global_stats)
    save_scaler(scaler)

    print("\nLoading TESTING Data...")
    X_test, y_test = load_data_from_csvs(test_csvs, global_stats, desc="Test Data", is_train=False, scaler=scaler)

    print(f"\nTrain shape: {X_train.shape}")
    print(f"Test shape:  {X_test.shape}")

    # =============================================================================
    # 6. APPLY SMOTE (CHỈ TRÊN TRAIN SET)
    # =============================================================================
    print("\n" + "="*70)
    print("APPLYING SMOTE TO BALANCE TRAINING DATA")
    print("="*70)
    
    print(f"\nPhân phối class TRƯỚC SMOTE: {Counter(y_train)}")
    
    # Áp dụng SMOTE để tăng số lượng mẫu Botnet
    # CHỈ áp dụng trên tập train, KHÔNG đụng vào test/val
    smote = SMOTE(random_state=42, k_neighbors=5, sampling_strategy='auto')
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    print(f"Phân phối class SAU SMOTE: {Counter(y_train_res)}")
    print(f"Train shape sau SMOTE: {X_train_res.shape}")
    print("="*70)

    # =============================================================================
    # 7. MODEL SET UP
    # =============================================================================
    
    # Validation Split (sau khi đã áp dụng SMOTE)
    X_train_final, X_val, y_train_final, y_val = train_test_split(
        X_train_res, y_train_res, test_size=0.2, random_state=42, stratify=y_train_res
    )

    train_ds = FastBotnetDataset(X_train_final, y_train_final)
    
    print("\nSử dụng CrossEntropyLoss (không dùng class_weight vì đã có SMOTE)")
   
    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0 # Avoid multiprocessing issues in some envs
    )

    valid_ds = FastBotnetDataset(X_val, y_val)
    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    test_ds = FastBotnetDataset(X_test, y_test)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # Model
    print("Initializing 2D CNN Image Model (pretrained)...")
    model = BotnetClassifier(base_model=None, n_features=n_features, n_classes=len(CLASS_TO_IDX))
    model = model.to(device)

    if summary:
        # Input: ảnh 1x32x32
        summary(model, input_size=(BATCH_SIZE, 1, IMAGE_SIZE, IMAGE_SIZE))
    else:
        print(model)

    # Bỏ class_weight vì đã dùng SMOTE để cân bằng dữ liệu
    # SMOTE đã tạo ra dữ liệu thật (synthetic) nên không cần ép model bằng trọng số
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    # =============================================================================
    # 8. TRAINING LOOP
    # =============================================================================
    train_losses = []
    valid_losses = []
    
    best_val_loss = float('inf')

    print(f"Starting training on {device}...")
    
    for epoch in range(1, N_EPOCHS + 1):
        model.train()
        running_loss = 0.0
        
    # Training
        for X_batch, y_batch in tqdm(train_loader, desc=f"Ep {epoch}/{N_EPOCHS} [Train]", leave=False):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * X_batch.size(0)
            
        epoch_train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in tqdm(valid_loader, desc=f"Ep {epoch}/{N_EPOCHS} [Val]", leave=False):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item() * X_batch.size(0)
                
        epoch_val_loss = val_loss / len(valid_loader.dataset)
        valid_losses.append(epoch_val_loss)
        
        print(f"Epoch {epoch}: Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")
        
        # Checkpoint
        if epoch_val_loss < best_val_loss:
            print(f"  Val loss decreased ({best_val_loss:.4f} -> {epoch_val_loss:.4f}). Saving model...")
            torch.save(model.state_dict(), 'best_model.pth')
            best_val_loss = epoch_val_loss
        plot_and_save_loss(train_losses, valid_losses, f'training_history_loss_{N_EPOCHS}.png')
    # =============================================================================
    # 9. RESULTS
    # =============================================================================
    print("Training Complete.")

# if __name__ == "__main__": # Block disabled for notebook import
#     train_main(ispart=True)


In [ ]:
# ==========================================
# CONTENT FROM: evaluate.py
# ==========================================
import os
import argparse
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import RobustScaler
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import gc

# from config import (  # Commented out for notebook compatibility
    WORKING_DIR, BATCH_SIZE, TRAIN_SCENARIOS, TEST_SCENARIOS,
    CLASS_TO_IDX, IDX_TO_CLASS, device
)
# from utils import get_csv_paths, create_directory, check_csv_in_folder, download_file, rename  # Commented out for notebook compatibility
# from preprocessing_utils import (  # Commented out for notebook compatibility
    quick_classify, calculate_global_frequencies, process_batch_fast_v2,
    save_global_stats, load_global_stats, save_scaler, load_scaler
)
# from model import BotnetClassifier  # Commented out for notebook compatibility
# from data_loader import FastBotnetDataset, load_data_from_csvs  # Commented out for notebook compatibility

def compute_stats_from_train(main_dir):
    """
    Recomputes global stats and scaler from training scenarios.
    """
    print("Recomputing statistics from training data...")
    train_csvs = get_csv_paths(main_dir, TRAIN_SCENARIOS)

    if not train_csvs:
        print("No training files found! Please check TRAIN_SCENARIOS and dataset.")
        return None, None

    # --- Step 1: Calculate Global Frequencies ---
    print("\n[1/2] Calculating global IP/Port frequencies...")
    freq_dicts = calculate_global_frequencies(train_csvs)

    # --- Step 2: Detect Column Schema ---
    print("\n[2/2] Detecting column schema...")
    expected_columns = None
    cols_samples = []

    for csv_file in train_csvs[:5]:
        try:
            chunk = pd.read_csv(csv_file, nrows=5000, low_memory=False)
            X_s, y_s, cols_s = process_batch_fast_v2(chunk, freq_dicts, expected_columns=None)
            if cols_s:
                cols_samples.extend(cols_s)
        except Exception:
            continue

    if cols_samples:
        expected_columns = list(dict.fromkeys(cols_samples))
        print(f"  Detected {len(expected_columns)} feature columns")
    else:
        print("  WARNING: Could not detect column schema!")
        return None, None

    global_stats = {
        'freq_dicts': freq_dicts,
        'expected_columns': expected_columns,
        'n_features': len(expected_columns)
    }

    # --- Step 4: Fit Scaler ---
    print("\nFitting Scaler on Training Data...")
    scaler = RobustScaler()
    # We need to load training data to fit scaler. 
    # This might be heavy, but necessary for correct evaluation if stats are missing.
    X_train, _ = load_data_from_csvs(train_csvs, global_stats, desc="Fitting Scaler", is_train=True, scaler=scaler)
    
    # Save for future
    save_global_stats(global_stats)
    save_scaler(scaler)
    
    # Clean up RAM
    del X_train
    gc.collect()

    return global_stats, scaler

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for X_batch, y_batch in tqdm(test_loader, desc="Evaluating"):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
            
    return np.array(all_labels), np.array(all_preds)

def plot_confusion_matrix_vietnamese(cm, classes, save_path):
    plt.figure(figsize=(10, 8))
    
    # Map class names to Vietnamese if desired, or keep English but title in VN
    # Common mappings: Normal -> Bình thường, Botnet -> Botnet, C&C -> C&C
    # Let's keep class names technical but title/labels in VN.
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Ma trận nhầm lẫn (Confusion Matrix)', fontsize=16)
    plt.ylabel('Nhãn thực tế (True Label)', fontsize=12)
    plt.xlabel('Nhãn dự đoán (Predicted Label)', fontsize=12)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def evaluate_main():
    parser = argparse.ArgumentParser(description="Evaluate Botnet Detection Model")
    parser.add_argument('--recompute-stats', action='store_true', help="Recompute global stats and scaler from training data instead of loading.")
    args = parser.parse_args()

    print(f"Device: {device}")
    
    # Check/Download Data (Reuse logic if needed, assuming data exists in CTU-13-Dataset)
    main_dir = './CTU-13-Dataset/'
    if not os.path.exists(main_dir):
        print("Dataset directory not found. Please run train.py to download data first.")
        return

    # 1. Load Stats and Scaler
    if args.recompute_stats or not os.path.exists('global_stats.pkl') or not os.path.exists('scaler.pkl'):
        global_stats, scaler = compute_stats_from_train(main_dir)
        if global_stats is None:
            return
    else:
        print("Loading existing statistics and scaler...")
        global_stats = load_global_stats()
        scaler = load_scaler()
    
    if global_stats is None:
        print("Failed to obtain global statistics.")
        return

    # 2. Load Test Data
    test_csvs = get_csv_paths(main_dir, TEST_SCENARIOS)
    print(f"Found {len(test_csvs)} testing files for evaluation.")
    
    # Load data
    X_test, y_test = load_data_from_csvs(test_csvs, global_stats, desc="Loading Test Data", is_train=False, scaler=scaler)

    print(f"Test Data Shape: {X_test.shape}")

    # 3. Load Model
    model_path = 'best_model.pth'
    if not os.path.exists(model_path):
        print(f"Model file {model_path} not found!")
        return
        
    # n_features được giữ cho tương thích API nhưng không dùng trong CNN ảnh
    model = BotnetClassifier(base_model=None, n_features=global_stats['n_features'], n_classes=len(CLASS_TO_IDX))
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    print("Model loaded successfully.")

    # 4. Evaluate
    test_ds = FastBotnetDataset(X_test, y_test)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0) # 0 workers for safety
    
    y_true, y_pred = evaluate_model(model, test_loader, device)

    # Quick sanity check: đếm số mẫu theo lớp trong ground-truth và dự đoán
    label_indices = list(range(len(CLASS_TO_IDX)))
    true_counts = {IDX_TO_CLASS[i]: int((y_true == i).sum()) for i in label_indices}
    pred_counts = {IDX_TO_CLASS[i]: int((y_pred == i).sum()) for i in label_indices}
    print("\nPhân bố nhãn (ground-truth):", true_counts)
    print("Phân bố nhãn (dự đoán):     ", pred_counts)

    # 5. Metrics & Reporting (Vietnamese)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # Use a fixed label order based on CLASS_TO_IDX so that
    # confusion matrix and classification report are consistent
    label_indices = list(range(len(CLASS_TO_IDX)))
    cm = confusion_matrix(y_true, y_pred, labels=label_indices)
    
    # Prepare output directory
    metrics_dir = 'metrics'
    create_directory(metrics_dir)
    
    report_path = os.path.join(metrics_dir, 'evaluation_report.txt')
    cm_path = os.path.join(metrics_dir, 'confusion_matrix.png')
    
    report_content = [
        "BÁO CÁO ĐÁNH GIÁ MÔ HÌNH (MODEL EVALUATION REPORT)",
        "="*50,
        f"Kịch bản kiểm thử (Test Scenarios): {TEST_SCENARIOS}",
        f"Tổng số mẫu (Total Samples): {len(y_true)}",
        "-"*50,
        f"Độ chính xác (Accuracy): {accuracy:.4f}",
        f"Độ chính xác (Precision - Weighted): {precision:.4f}",
        f"Độ nhạy (Recall - Weighted): {recall:.4f}",
        f"Điểm F1 (F1-Score - Weighted): {f1:.4f}",
        "="*50,
        "Chi tiết theo lớp (Class-wise Details):"
    ]
    
    # Class-wise metrics
    # Re-map indices to class names (must match label_indices order)
    classes = [IDX_TO_CLASS[i] for i in label_indices]
    
    # Calculate per-class precision/recall/f1.
    # Explicitly pass labels so that number of labels matches target_names,
    # even if some classes do not appear in y_true/y_pred.
    from sklearn.metrics import classification_report
    cls_report = classification_report(
        y_true,
        y_pred,
        labels=label_indices,
        target_names=classes,
        output_dict=True,
        zero_division=0,
    )
    
    for cls in classes:
        metrics = cls_report.get(cls, {})
        report_content.append(f"\nLớp: {cls}")
        report_content.append(f"  Precision: {metrics.get('precision', 0):.4f}")
        report_content.append(f"  Recall:    {metrics.get('recall', 0):.4f}")
        report_content.append(f"  F1-Score:  {metrics.get('f1-score', 0):.4f}")
        
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report_content))
        
    print(f"\nReport saved to {report_path}")
    print('\n'.join(report_content))

    # Plot Confusion Matrix
    plot_confusion_matrix_vietnamese(cm, classes, cm_path)
    print(f"Confusion Matrix saved to {cm_path}")

# if __name__ == "__main__": # Block disabled for notebook import
#     evaluate_main()


In [ ]:
# ==========================================
# EXECUTION: TRAIN
# ==========================================
if __name__ == '__main__':
    print('Starting Training...')
    # Set ispart=False for full training
    train_main(ispart=True)


In [ ]:
# ==========================================
# EXECUTION: EVALUATE
# ==========================================
import sys
# Simulate command line arguments. Use empty list for defaults.
# To recompute stats: sys.argv = ['evaluate.py', '--recompute-stats']
sys.argv = ['evaluate.py'] 

if __name__ == '__main__':
    print('Starting Evaluation...')
    evaluate_main()


In [ ]:
# ==========================================
# EXECUTION: ANALYZE CSV (Example)
# ==========================================
# sys.argv = ['analyze_csv.py', '--file', './CTU-13-Dataset/1/capture20110810.csv']
# analyze_csv_main()
